In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from atria_models.pipelines.atria_model_pipeline import AtriaModelPipeline
from atria_models.registry import MODEL_PIPELINE
import json
from atria_core.types import DatasetMetadata, DatasetLabels

model_pipeline: AtriaModelPipeline = AtriaModelPipeline.load_from_registry(
    "image_classification",
    model_name="resnet50",
    dataset_metadata=DatasetMetadata(
        dataset_labels=DatasetLabels(classification=[str(i) for i in range(10)])
    ),
    overrides=["model_pipeline.model.model_name=resnet50"],
    provider="atria_models",
    search_pkgs=["atria_models", "atria_transforms"],
)
print(model_pipeline)

[2025-07-20 17:59:43][atria_models.core.timm_model][INFO] Setting up model TimmModel(model_name='resnet50', num_classes=10)


ImageClassificationPipeline:
Layer (type:depth-idx)                             Param #
ModuleDict                                         --
├─TimmModel: 1-1                                   --
│    └─ResNet: 2-1                                 --
│    │    └─Conv2d: 3-1                            9,408
│    │    └─BatchNorm2d: 3-2                       128
│    │    └─ReLU: 3-3                              --
│    │    └─MaxPool2d: 3-4                         --
│    │    └─Sequential: 3-5                        215,808
│    │    └─Sequential: 3-6                        1,219,584
│    │    └─Sequential: 3-7                        7,098,368
│    │    └─Sequential: 3-8                        14,964,736
│    │    └─SelectAdaptivePool2d: 3-9              --
│    │    └─Linear: 3-10                           20,490
├─CrossEntropyLoss: 1-2                            --
├─CrossEntropyLoss: 1-3                            --
Total params: 23,528,522
Trainable params: 23,528,522
Non-trainable

In [7]:
model_pipeline.to_device()
print(model_pipeline.model._model.conv1.weight.device)

[2025-07-20 17:59:43][atria_models.utilities.nn_modules][INFO] Moving model to available device: cpu


cpu


In [8]:
acc = model_pipeline.metrics['accuracy']
print(acc.__dict__)
# print(acc)
# from ignite.metrics import Accuracy
# acc.to('cpu')

{'_is_multilabel': False, '_type': None, '_num_classes': None, '_output_transform': <function _output_transform at 0x71e496061080>, '_device': device(type='cpu'), '_skip_unrolling': False, '_double_dtype': torch.float64, '_num_correct': tensor(0), '_num_examples': 0}


In [9]:
from atria_core.types.factory import ImageInstanceFactory
from atria_core.types import GroundTruth
model_pipeline.enable_runtime_transforms()
image = ImageInstanceFactory.build()
image.gt = GroundTruth(classification = image.gt.classification)
model_pipeline.training_step([image])

[4, 10, 2, 10, 10, 8, 10, 6, 2, 5]
[9, 3, 0, 10, 5, 7, 1, 3, 7, 0]


ClassificationModelOutput(loss=tensor(2.3843, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0123, -0.0721,  0.2128,  0.3599,  0.1587,  0.1840, -0.0034,  0.3808,
         -0.3062,  0.0308]], grad_fn=<AddmmBackward0>), prediction=None, label=Label(value=tensor([9]), name=['interesting']))

ModuleNotFoundError: No module named 'atria_ml'

In [11]:
# from atria_models.pipelines.classification.image import ImageClassificationPipeline

# print(model_pipeline.config)
# model_pipeline.upload_to_hub(
#     "mymodel"
# )

# loaded = ImageClassificationPipeline.load_from_hub(
#     "mymodel", 
# )

In [12]:
# loaded.state_dict()['dataset_metadata'] = model_pipeline.state_dict()['dataset_metadata']
# loaded.state_dict()['config'] = model_pipeline.state_dict()['config']
# print(loaded.state_dict()['config'])

In [13]:
# from atria_models.pipelines.classification.image import ImageClassificationPipeline
# from omegaconf import OmegaConf
# from atria_models.registry import MODEL
# from atria_models.core.timm_model import TimmModel
# from rich.pretty import pretty_repr
# import json
# model = TimmModel(model_name="resnet50")
# pipeline = ImageClassificationPipeline(model=model, checkpoint_configs='test')
# print(json.dumps(pipeline._config, indent=2))